# How To: Manage Azure Sentinel Bookmarks

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML), Plotly 3.5 <br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The sample notebook get bookmarks from Azure Sentinel

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Retrieve Log Analytics information1. 
2. Bookmark functions 
3. Create bookmark and get bookmarks
4. Go to Azure Log Analytics
5. Data Analysis - Timeline Chart

## 1. Retrieve Log Analytics Information

In [ ]:
import json
def read_config_values(filePath):
    with open(filePath) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config["tenant_id"],
                    json_config["subscription_id"],
                    json_config["resource_group"],
                    json_config["workspace_id"],
                    json_config["workspace_name"])
    return None

path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    # Run this if you are using Free Compute
    workspace_id = read_config_values("../config.json")[3]
    subscription_id = read_config_values("../config.json")[1]
    resource_group = read_config_values("../config.json")[2]

    ######################################################################
    workspace_name = read_config_values("../config.json")[4]
    print("Your Log Analytic Workspace: " + workspace_name)
    ######################################################################
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')

## 2. Bookmark functions

In [ ]:
# head
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

# calling test site
test_subscription_id = input('test subscription id:')
test_resource_group = input('test resource_group:')
test_workspace_name = input('test workspace_name:')

In [ ]:
# functions, this is using testing environment
def generate_guid():
    import uuid
    return str(uuid.uuid4())

def construct_url(bookmark_id):
    base_url = 'https://resourceprovider.westus2.cloudapp.azure.com/subscriptions/{}/resourceGroups/{}/providers/Microsoft.OperationalInsights/workspaces/{}/providers/Microsoft.SecurityInsights/bookmarks{}?api-version=1.0'
    return base_url.format(test_subscription_id, test_resource_group, test_workspace_name, '/' + bookmark_id) if bookmark_id != None else base_url.format(test_subscription_id, test_resource_group, test_workspace_name, '')

def create_bookmark():
    import datetime
    # Sample bookmark data
    newbookmark = {
      "properties": {
        "displayName": "Example bookmark",
        "notes": datetime.datetime.now().isoformat(),
        "labels": ["Azure Sentinel Notebooks"],
        "query": "SecurityEvent | take 5 | project Account, Computer",
        "queryResult": "'Account': 'WORKGROUP\vm3$', 'Computer': 'vm3'"
      }
    }
    
    url = construct_url(generate_guid())
    response = None
    headers = {"Content-Type" : "application/json"}
    try:
        response = requests.put(url, data=json.dumps(newbookmark), headers=headers)

        if response.status_code != 200:
            print(response.status_code)
            print(response.text)
            raise Exception('Recieved non 200 response while sending response to CFN.')
        return response
    except requests.exceptions.RequestException as e:
        if response != None:
            print(response.text)
        print(e)
        raise
        
def get_bookmarks():
    url = construct_url(None)
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    resJson = json.loads(response.text)
    if resJson != None:
        try:
            return json_normalize(resJson['value'])
        except Exception as e:
            print(e)

## 3. Create bookmark and get bookmarks

In [ ]:
# Create a new bookmark
res = create_bookmark()
print(res)

In [ ]:
# get all bookamrks and filter by displayName
origin_df = get_bookmarks()
df = origin_df[origin_df['properties.displayName'] == 'Example bookmark'][['properties.created', 'properties.notes', 'properties.displayName', 'properties.labels', 'properties.query', 'properties.queryResult']]
df

## 4. Go to Azure Log Analytics

In [ ]:
# Utils functions
def copy_query_to_clipboard(query_text):
    import ipywidgets as widgets
    from IPython.display import display
    from IPython.display import HTML

    url = 'https://ms.portal.azure.com/?feature.showassettypes=Microsoft_Azure_Security_Insights_SecurityInsightsDashboard#blade/Microsoft_Azure_Security_Insights/MainMenuBlade/7/subscriptionId/{}/resourceGroup/{}/workspaceName/{}'.format(subscription_id, resource_group, workspace_name)
    html_str = (
        """<!DOCTYPE html>
        <html><body>

        <input  id="asi_demo_query" type="text" readonly style="font-weight: bold; border: none; width:1px;" size = '"""
        + str(len(query_text))
        + """' value='"""
        + query_text
        + """'>

        <a target="_new" href="javascript:void(0);" onclick="asi_demo_copy()">Go to Log Analytics</a>
    
        <script>
        var asi_demo_win = null
        function asi_demo_copy() {
            var copyText = document.getElementById("asi_demo_query");
            copyText.select();
            document.execCommand("copy");

            var w = screen.width - 300;
            var h = screen.height - 300;
            params = 'width='+w+',height='+h
            asi_demo_win = window.open('"""
                    + url
                    + """', 'asi_demo_win', params);
        }

        </script>
        </body></html>"""
    )

    return html_str    

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_new" href="{}"></a>'.format(val)

In [ ]:
# appending link column to the dataframe
df = df.assign(GoToLogAnalytics=df['properties.query'].apply(lambda x: copy_query_to_clipboard(x)))
df.style.format({'': make_clickable})

## 5. Data Analysis - Timeline Chart

In [ ]:
origin_df['Start'] = origin_df.groupby('properties.displayName')['properties.created'].transform(min)
origin_df['Finish'] = origin_df.groupby('properties.displayName')['properties.created'].transform(max)

plot_df = origin_df[['properties.displayName', 'Start', 'Finish']].copy().rename(columns = {'properties.displayName' : 'Task'}).drop_duplicates('Task')
plot_df = plot_df.reset_index().drop('index', 1)
plot_df

In [ ]:
import plotly
import plotly.plotly as pplotly
import plotly.figure_factory as factory

# input your account info for Plotly, https://plot.ly
your_plotly_username = input('plotly username:')
your_api_key = input('plotly API key:')
plotly.tools.set_credentials_file(username=your_plotly_username, api_key=your_api_key)

In [ ]:
chart = factory.create_gantt(plot_df)
pplotly.iplot(chart, filename='Bookmark Time Line', title='Bookmark Time Line', world_readable=True)